<a href="https://colab.research.google.com/github/nlemoff/data102project/blob/main/Checkpoint2DataSourcing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
# Fetch electricity demand data
electricity_url = 'https://api.eia.gov/v2/electricity/retail-sales/data/?frequency=monthly&data[0]=sales&facets[stateid][]=CA'
electricity_response = requests.get(electricity_url, params={'api_key': eia_api_key})

if electricity_response.status_code == 200:
    electricity_data = pd.DataFrame(electricity_response.json()['response']['data'])
    print("Electricity demand data successfully retrieved. Shape:", electricity_data.shape)
    print(electricity_data.head())
else:
    print(f"Error fetching electricity demand data. Status code: {electricity_response.status_code}")


Electricity demand data successfully retrieved. Shape: (1710, 7)
    period stateid stateDescription sectorid   sectorName        sales  \
0  2024-03      CA       California      ALL  all sectors  17659.18439   
1  2024-03      CA       California      COM   commercial   8749.35296   
2  2024-03      CA       California      IND   industrial    2927.1454   
3  2024-03      CA       California      OTH        other         None   
4  2024-03      CA       California      RES  residential   5918.23144   

              sales-units  
0  million kilowatt hours  
1  million kilowatt hours  
2  million kilowatt hours  
3  million kilowatt hours  
4  million kilowatt hours  


In [35]:
# Fetch emissions data
emissions_url = 'https://api.eia.gov/v2/electricity/state-electricity-profiles/emissions-by-state-by-fuel/data/?frequency=annual&data[0]=co2-thousand-metric-tons'
emissions_response = requests.get(emissions_url, params={'api_key': eia_api_key})

if emissions_response.status_code == 200:
    emissions_data = pd.DataFrame(emissions_response.json()['response']['data'])
    print("Emissions data successfully retrieved. Shape:", emissions_data.shape)
    print(emissions_data.head())
else:
    print(f"Error fetching emissions data. Status code: {emissions_response.status_code}")


Emissions data successfully retrieved. Shape: (5000, 7)
  period stateid stateDescription fuelid fuelDescription  \
0   2017      AR         Arkansas    PET       Petroleum   
1   2017      AR         Arkansas    ALL           Total   
2   1993      AR         Arkansas    COL            Coal   
3   1993      AR         Arkansas     NG     Natural Gas   
4   1993      AR         Arkansas    OTH           Other   

  co2-thousand-metric-tons co2-thousand-metric-tons-units  
0                       43           thousand metric tons  
1                    33322           thousand metric tons  
2                    18984           thousand metric tons  
3                     2623           thousand metric tons  
4                       74           thousand metric tons  


In [44]:
import pandas as pd
import requests

# NOAA API base URL and token
base_url = "https://www.ncei.noaa.gov/cdo-web/api/v2/data"
api_token = "qhmRiNPKavISCVDDZzzxzTxBnWuhOLBp"

# Headers and initial parameters
headers = {"token": api_token}
params = {
    "datasetid": "GHCND",  # Global Historical Climatology Network Daily
    "datatypeid": "TAVG",  # Average temperature
    "locationid": "FIPS:06",  # California
    "startdate": "2023-01-01",  # Start fetching from the start of the year
    "enddate": "2023-12-31",    # End of the year
    "limit": 1000,              # Maximum number of records per request
}

# Initialize an empty DataFrame to store results
temperature_data = pd.DataFrame()

while True:
    # Fetch data
    response = requests.get(base_url, headers=headers, params=params)
    if response.status_code != 200:
        print(f"Error: Received status code {response.status_code}")
        break

    data = response.json()
    results = data.get('results', [])
    if not results:  # If no results, stop fetching
        print("No more data to fetch.")
        break

    # Convert to DataFrame and append to the main dataset
    batch_data = pd.DataFrame(results)
    temperature_data = pd.concat([temperature_data, batch_data], ignore_index=True)

    # Update `startdate` to the day after the latest fetched date
    latest_date = pd.to_datetime(batch_data['date']).max()
    next_start_date = latest_date + pd.Timedelta(days=1)

    # Stop if the latest date exceeds the desired range
    if next_start_date > pd.Timestamp("2023-12-31"):
        print("Data fetching complete.")
        break

    # Update the `startdate` parameter
    params['startdate'] = next_start_date.strftime("%Y-%m-%d")

    print(f"Fetched up to {latest_date}. Continuing from {next_start_date}...")

# Final dataset verification
print("Final dataset shape:", temperature_data.shape)
print(temperature_data.head())


Fetched up to 2023-01-04 00:00:00. Continuing from 2023-01-05 00:00:00...
Fetched up to 2023-01-08 00:00:00. Continuing from 2023-01-09 00:00:00...
Fetched up to 2023-01-12 00:00:00. Continuing from 2023-01-13 00:00:00...
Fetched up to 2023-01-16 00:00:00. Continuing from 2023-01-17 00:00:00...
Fetched up to 2023-01-20 00:00:00. Continuing from 2023-01-21 00:00:00...
Fetched up to 2023-01-24 00:00:00. Continuing from 2023-01-25 00:00:00...
Fetched up to 2023-01-28 00:00:00. Continuing from 2023-01-29 00:00:00...
Fetched up to 2023-02-01 00:00:00. Continuing from 2023-02-02 00:00:00...
Fetched up to 2023-02-05 00:00:00. Continuing from 2023-02-06 00:00:00...
Fetched up to 2023-02-09 00:00:00. Continuing from 2023-02-10 00:00:00...
Fetched up to 2023-02-13 00:00:00. Continuing from 2023-02-14 00:00:00...
Fetched up to 2023-02-17 00:00:00. Continuing from 2023-02-18 00:00:00...
Fetched up to 2023-02-21 00:00:00. Continuing from 2023-02-22 00:00:00...
Fetched up to 2023-02-25 00:00:00. Con

In [46]:
temperature_data

,date,datatype,station,attributes,value
0,2023-01-01T00:00:00,TAVG,GHCND:USR0000CABS,",,U,",64
1,2023-01-01T00:00:00,TAVG,GHCND:USR0000CACT,",,U,",72
2,2023-01-01T00:00:00,TAVG,GHCND:USR0000CALD,",,U,",28
3,2023-01-01T00:00:00,TAVG,GHCND:USR0000CALP,",,U,",69
4,2023-01-01T00:00:00,TAVG,GHCND:USR0000CALT,",,U,",110
...,...,...,...,...,...
58995,2023-08-24T00:00:00,TAVG,GHCND:USR0000CCLK,",,U,",246
58996,2023-08-24T00:00:00,TAVG,GHCND:USR0000CCLV,",,U,",224
58997,2023-08-24T00:00:00,TAVG,GHCND:USR0000CCMO,",,U,",198
58998,2023-08-24T00:00:00,TAVG,GHCND:USR0000CCOH,",,U,",288


In [47]:
import pandas as pd
import requests

# NOAA API base URL and token
base_url = "https://www.ncei.noaa.gov/cdo-web/api/v2/data"
api_token = "qhmRiNPKavISCVDDZzzxzTxBnWuhOLBp"

# Headers and parameters
headers = {"token": api_token}
params = {
    "datasetid": "GHCND",  # Global Historical Climatology Network Daily
    "datatypeid": "TAVG",  # Average temperature
    "locationid": "FIPS:06",  # California
    "startdate": "2023-08-25",  # Start fetching from August 25
    "enddate": "2023-12-31",    # End of the year
    "limit": 1000,              # Maximum number of records per request
}

# Initialize an empty DataFrame for new data
new_temperature_data = pd.DataFrame()

while True:
    # Fetch data
    response = requests.get(base_url, headers=headers, params=params)
    if response.status_code != 200:
        print(f"Error: Received status code {response.status_code}")
        break

    data = response.json()
    results = data.get('results', [])
    if not results:  # Stop if no more results
        print("No more data to fetch.")
        break

    # Convert to DataFrame and append to the new dataset
    batch_data = pd.DataFrame(results)
    new_temperature_data = pd.concat([new_temperature_data, batch_data], ignore_index=True)

    # Update `startdate` to the day after the latest fetched date
    latest_date = pd.to_datetime(batch_data['date']).max()
    next_start_date = latest_date + pd.Timedelta(days=1)

    # Stop if the latest date exceeds the desired range
    if next_start_date > pd.Timestamp("2023-12-31"):
        print("Data fetching complete.")
        break

    # Update the `startdate` parameter
    params['startdate'] = next_start_date.strftime("%Y-%m-%d")

    print(f"Fetched up to {latest_date}. Continuing from {next_start_date}...")

# Final dataset verification
print("Final dataset shape:", new_temperature_data.shape)
print(new_temperature_data.head())


Fetched up to 2023-08-28 00:00:00. Continuing from 2023-08-29 00:00:00...
Fetched up to 2023-09-01 00:00:00. Continuing from 2023-09-02 00:00:00...
Fetched up to 2023-09-05 00:00:00. Continuing from 2023-09-06 00:00:00...
Fetched up to 2023-09-09 00:00:00. Continuing from 2023-09-10 00:00:00...
Fetched up to 2023-09-13 00:00:00. Continuing from 2023-09-14 00:00:00...
Fetched up to 2023-09-17 00:00:00. Continuing from 2023-09-18 00:00:00...
Fetched up to 2023-09-21 00:00:00. Continuing from 2023-09-22 00:00:00...
Fetched up to 2023-09-25 00:00:00. Continuing from 2023-09-26 00:00:00...
Fetched up to 2023-09-29 00:00:00. Continuing from 2023-09-30 00:00:00...
Fetched up to 2023-10-03 00:00:00. Continuing from 2023-10-04 00:00:00...
Fetched up to 2023-10-07 00:00:00. Continuing from 2023-10-08 00:00:00...
Fetched up to 2023-10-11 00:00:00. Continuing from 2023-10-12 00:00:00...
Fetched up to 2023-10-15 00:00:00. Continuing from 2023-10-16 00:00:00...
Fetched up to 2023-10-19 00:00:00. Con

In [74]:
new_temperature_data.to_csv('missing.csv', index=False)


In [49]:
import pandas as pd
import requests

# NOAA API base URL and token
base_url = "https://www.ncei.noaa.gov/cdo-web/api/v2/data"
api_token = "qhmRiNPKavISCVDDZzzxzTxBnWuhOLBp"

# Headers and parameters
headers = {"token": api_token}
params = {
    "datasetid": "GHCND",  # Global Historical Climatology Network Daily
    "datatypeid": "TAVG",  # Average temperature
    "locationid": "FIPS:06",  # California
    "startdate": "2023-11-13",  # Start fetching from November 13
    "enddate": "2023-12-31",    # End of the year
    "limit": 1000,              # Maximum number of records per request
}

# Initialize an empty DataFrame for new data
temperature_data_11_13_onward = pd.DataFrame()

while True:
    # Fetch data
    response = requests.get(base_url, headers=headers, params=params)
    if response.status_code != 200:
        print(f"Error: Received status code {response.status_code}")
        break

    data = response.json()
    results = data.get('results', [])
    if not results:  # Stop if no more results
        print("No more data to fetch.")
        break

    # Convert to DataFrame and append to the new dataset
    batch_data = pd.DataFrame(results)
    temperature_data_11_13_onward = pd.concat([temperature_data_11_13_onward, batch_data], ignore_index=True)

    # Update `startdate` to the day after the latest fetched date
    latest_date = pd.to_datetime(batch_data['date']).max()
    next_start_date = latest_date + pd.Timedelta(days=1)

    # Stop if the latest date exceeds the desired range
    if next_start_date > pd.Timestamp("2023-12-31"):
        print("Data fetching complete.")
        break

    # Update the `startdate` parameter
    params['startdate'] = next_start_date.strftime("%Y-%m-%d")

    print(f"Fetched up to {latest_date}. Continuing from {next_start_date}...")

# Final dataset verification
print("Final dataset shape:", temperature_data_11_13_onward.shape)
print(temperature_data_11_13_onward.head())


Fetched up to 2023-11-16 00:00:00. Continuing from 2023-11-17 00:00:00...
Fetched up to 2023-11-20 00:00:00. Continuing from 2023-11-21 00:00:00...
Error: Received status code 503
Final dataset shape: (2000, 5)
                  date datatype            station attributes  value
0  2023-11-13T00:00:00     TAVG  GHCND:USR0000CABS       ,,U,    118
1  2023-11-13T00:00:00     TAVG  GHCND:USR0000CACT       ,,U,    123
2  2023-11-13T00:00:00     TAVG  GHCND:USR0000CALD       ,,U,     78
3  2023-11-13T00:00:00     TAVG  GHCND:USR0000CALP       ,,U,    157
4  2023-11-13T00:00:00     TAVG  GHCND:USR0000CALT       ,,U,    132


In [50]:
temperature_data_11_13_onward

,date,datatype,station,attributes,value
0,2023-11-13T00:00:00,TAVG,GHCND:USR0000CABS,",,U,",118
1,2023-11-13T00:00:00,TAVG,GHCND:USR0000CACT,",,U,",123
2,2023-11-13T00:00:00,TAVG,GHCND:USR0000CALD,",,U,",78
3,2023-11-13T00:00:00,TAVG,GHCND:USR0000CALP,",,U,",157
4,2023-11-13T00:00:00,TAVG,GHCND:USR0000CALT,",,U,",132
...,...,...,...,...,...
1995,2023-11-20T00:00:00,TAVG,GHCND:USR0000CCLE,",,U,",149
1996,2023-11-20T00:00:00,TAVG,GHCND:USR0000CCLK,",,U,",164
1997,2023-11-20T00:00:00,TAVG,GHCND:USR0000CCLV,",,U,",114
1998,2023-11-20T00:00:00,TAVG,GHCND:USR0000CCMO,",,U,",98


In [51]:
import pandas as pd
import requests

# NOAA API base URL and token
base_url = "https://www.ncei.noaa.gov/cdo-web/api/v2/data"
api_token = "qhmRiNPKavISCVDDZzzxzTxBnWuhOLBp"

# Headers and parameters
headers = {"token": api_token}
params = {
    "datasetid": "GHCND",  # Global Historical Climatology Network Daily
    "datatypeid": "TAVG",  # Average temperature
    "locationid": "FIPS:06",  # California
    "startdate": "2023-11-21",  # Start fetching from November 21
    "enddate": "2023-12-31",    # End of the year
    "limit": 1000,              # Maximum number of records per request
}

# Initialize an empty DataFrame for new data
temperature_data_11_21_onward = pd.DataFrame()

while True:
    # Fetch data
    response = requests.get(base_url, headers=headers, params=params)
    if response.status_code != 200:
        print(f"Error: Received status code {response.status_code}")
        break

    data = response.json()
    results = data.get('results', [])
    if not results:  # Stop if no more results
        print("No more data to fetch.")
        break

    # Convert to DataFrame and append to the new dataset
    batch_data = pd.DataFrame(results)
    temperature_data_11_21_onward = pd.concat([temperature_data_11_21_onward, batch_data], ignore_index=True)

    # Update `startdate` to the day after the latest fetched date
    latest_date = pd.to_datetime(batch_data['date']).max()
    next_start_date = latest_date + pd.Timedelta(days=1)

    # Stop if the latest date exceeds the desired range
    if next_start_date > pd.Timestamp("2023-12-31"):
        print("Data fetching complete.")
        break

    # Update the `startdate` parameter
    params['startdate'] = next_start_date.strftime("%Y-%m-%d")

    print(f"Fetched up to {latest_date}. Continuing from {next_start_date}...")

# Final dataset verification
print("Final dataset shape:", temperature_data_11_21_onward.shape)
print(temperature_data_11_21_onward.head())


Fetched up to 2023-11-24 00:00:00. Continuing from 2023-11-25 00:00:00...
Fetched up to 2023-11-28 00:00:00. Continuing from 2023-11-29 00:00:00...
Fetched up to 2023-12-02 00:00:00. Continuing from 2023-12-03 00:00:00...
Fetched up to 2023-12-06 00:00:00. Continuing from 2023-12-07 00:00:00...
Fetched up to 2023-12-10 00:00:00. Continuing from 2023-12-11 00:00:00...
Fetched up to 2023-12-14 00:00:00. Continuing from 2023-12-15 00:00:00...
Fetched up to 2023-12-18 00:00:00. Continuing from 2023-12-19 00:00:00...
Fetched up to 2023-12-22 00:00:00. Continuing from 2023-12-23 00:00:00...
Fetched up to 2023-12-26 00:00:00. Continuing from 2023-12-27 00:00:00...
Fetched up to 2023-12-30 00:00:00. Continuing from 2023-12-31 00:00:00...
Data fetching complete.
Final dataset shape: (10046, 5)
                  date datatype            station attributes  value
0  2023-11-21T00:00:00     TAVG  GHCND:USR0000CABS       ,,U,    134
1  2023-11-21T00:00:00     TAVG  GHCND:USR0000CACT       ,,U,    1

In [52]:
# Combine all DataFrames into one
all_temperature_data = pd.concat(
    [temperature_data, temperature_data_11_13_onward, temperature_data_11_21_onward],
    ignore_index=True
)

# Ensure chronological order by sorting on the 'date' column
all_temperature_data['date'] = pd.to_datetime(all_temperature_data['date'])  # Ensure 'date' is a datetime type
all_temperature_data = all_temperature_data.sort_values(by='date').reset_index(drop=True)

# Verify combined DataFrame
print("Combined dataset shape:", all_temperature_data.shape)
print(all_temperature_data.head())
print(all_temperature_data.tail())


Combined dataset shape: (71046, 5)
        date datatype            station attributes  value
0 2023-01-01     TAVG  GHCND:USR0000CABS       ,,U,     64
1 2023-01-01     TAVG  GHCND:USR0000CSFD       ,,U,    123
2 2023-01-01     TAVG  GHCND:USR0000CSEC       ,,U,     45
3 2023-01-01     TAVG  GHCND:USR0000CSDD       ,,U,    -54
4 2023-01-01     TAVG  GHCND:USR0000CSDC       ,,U,     29
            date datatype            station attributes  value
71041 2023-12-31     TAVG  GHCND:USS0020H06S       ,,T,     -4
71042 2023-12-31     TAVG  GHCND:USS0020H12S       ,,T,    -28
71043 2023-12-31     TAVG  GHCND:USS0020K03S       ,,T,    -17
71044 2023-12-31     TAVG  GHCND:USS0019L39S       ,,T,      3
71045 2023-12-31     TAVG  GHCND:USW00093228      H,,S,    122


In [53]:
# Export the combined dataset to a CSV file
output_file = "temperature_data_2023.csv"
all_temperature_data.to_csv(output_file, index=False)

print(f"Data successfully exported to {output_file}")


Data successfully exported to temperature_data_2023.csv


In [65]:
# Electricity retail sales data for all sectors
URL = 'https://api.eia.gov/v2/electricity/retail-sales/data/?frequency=monthly&data[0]=sales&facets[stateid][]=CA&facets[sectorid][]=ALL&start=2023-01&end=2024-01&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000'
key = 'vgyKsY7pYEHEeo0dncg7ho5nlLp0h2JjAAMBT6nL'
data = requests.get(URL, params={'api_key': key}).json()
state_wise_energy = pd.DataFrame.from_dict(data['response']['data'])

# Save to CSV
state_wise_energy


,period,stateid,stateDescription,sectorid,sectorName,sales,sales-units
0,2024-01,CA,California,ALL,all sectors,20040.74935,million kilowatt hours
1,2023-12,CA,California,ALL,all sectors,18660.12634,million kilowatt hours
2,2023-11,CA,California,ALL,all sectors,18479.20564,million kilowatt hours
3,2023-10,CA,California,ALL,all sectors,20390.47299,million kilowatt hours
4,2023-09,CA,California,ALL,all sectors,22466.03859,million kilowatt hours
5,2023-08,CA,California,ALL,all sectors,25101.55563,million kilowatt hours
6,2023-07,CA,California,ALL,all sectors,23048.26218,million kilowatt hours
7,2023-06,CA,California,ALL,all sectors,19333.43379,million kilowatt hours
8,2023-05,CA,California,ALL,all sectors,18288.412,million kilowatt hours
9,2023-04,CA,California,ALL,all sectors,16261.33366,million kilowatt hours


In [55]:
# Electricity retail sales data for the residential sector
URL = 'https://api.eia.gov/v2/electricity/retail-sales/data/?frequency=monthly&data[0]=sales&facets[stateid][]=CA&facets[sectorid][]=RES&start=2023-01&end=2024-01&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000'
data = requests.get(URL, params={'api_key': key}).json()
state_wise_energy_res = pd.DataFrame.from_dict(data['response']['data'])

# Save to CSV
state_wise_energy_res.to_csv('state_wise_energy_residential.csv', index=False)


In [56]:
# Electricity retail sales data for the industrial sector
URL = 'https://api.eia.gov/v2/electricity/retail-sales/data/?frequency=monthly&data[0]=sales&facets[stateid][]=CA&facets[sectorid][]=IND&start=2023-01&end=2024-01&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000'
data = requests.get(URL, params={'api_key': key}).json()
state_wise_energy_ind = pd.DataFrame.from_dict(data['response']['data'])

# Save to CSV
state_wise_energy_ind.to_csv('state_wise_energy_industrial.csv', index=False)


In [62]:
# Electricity retail sales data for the transportation sector
URL = 'https://api.eia.gov/v2/electricity/retail-sales/data/?frequency=monthly&data[0]=sales&facets[stateid][]=CA&facets[sectorid][]=TRA&start=2023-01&end=2024-01&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000'
data = requests.get(URL, params={'api_key': key}).json()
state_wise_energy_tra = pd.DataFrame.from_dict(data['response']['data'])

# Save to CSV
state_wise_energy_tra


,period,stateid,stateDescription,sectorid,sectorName,sales,sales-units
0,2024-01,CA,California,TRA,transportation,51.74829,million kilowatt hours
1,2023-12,CA,California,TRA,transportation,51.26901,million kilowatt hours
2,2023-11,CA,California,TRA,transportation,58.99317,million kilowatt hours
3,2023-10,CA,California,TRA,transportation,62.01097,million kilowatt hours
4,2023-09,CA,California,TRA,transportation,60.92997,million kilowatt hours
5,2023-08,CA,California,TRA,transportation,68.17682,million kilowatt hours
6,2023-07,CA,California,TRA,transportation,62.11936,million kilowatt hours
7,2023-06,CA,California,TRA,transportation,62.24481,million kilowatt hours
8,2023-05,CA,California,TRA,transportation,71.4942,million kilowatt hours
9,2023-04,CA,California,TRA,transportation,56.53867,million kilowatt hours


In [67]:
# State-level electricity emissions data
URL = 'https://api.eia.gov/v2/electricity/state-electricity-profiles/emissions-by-state-by-fuel/data/?frequency=annual&data[0]=co2-rate-lbs-mwh&data[1]=co2-thousand-metric-tons&data[2]=nox-rate-lbs-mwh&data[3]=nox-short-tons&data[4]=so2-rate-lbs-mwh&data[5]=so2-short-tons&start=1990&end=2023&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000'
data = requests.get(URL, params={'api_key': key}).json()
state_emissions = pd.DataFrame.from_dict(data['response']['data'])

# Save to CSV
state_emissions


,period,stateid,stateDescription,fuelid,fuelDescription,co2-rate-lbs-mwh,co2-thousand-metric-tons,nox-rate-lbs-mwh,nox-short-tons,so2-rate-lbs-mwh,so2-short-tons,co2-rate-lbs-mwh-units,co2-thousand-metric-tons-units,nox-rate-lbs-mwh-units,nox-short-tons-units,so2-rate-lbs-mwh-units,so2-short-tons-units
0,2023,AZ,Arizona,COL,Coal,None,12863,None,9238,None,6854,pounds per megawatthour,thousand metric tons,pounds per megawatthour,short tons,pounds per megawatthour,short tons
1,2023,WY,Wyoming,ALL,Total,1864,36580,1.4,31178,1.1,24209,pounds per megawatthour,thousand metric tons,pounds per megawatthour,short tons,pounds per megawatthour,short tons
2,2023,WY,Wyoming,PET,Petroleum,None,37,None,22,None,13,pounds per megawatthour,thousand metric tons,pounds per megawatthour,short tons,pounds per megawatthour,short tons
3,2023,WY,Wyoming,OTH,Other,None,0,None,3436,None,8,pounds per megawatthour,thousand metric tons,pounds per megawatthour,short tons,pounds per megawatthour,short tons
4,2023,WY,Wyoming,NG,Natural Gas,None,1516,None,2489,None,9,pounds per megawatthour,thousand metric tons,pounds per megawatthour,short tons,pounds per megawatthour,short tons
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2004,CA,California,COL,Coal,None,3950,None,3455,None,2740,pounds per megawatthour,thousand metric tons,pounds per megawatthour,short tons,pounds per megawatthour,short tons
4996,2004,AZ,Arizona,ALL,Total,1113,52900,1.6,86019,1.2,60544,pounds per megawatthour,thousand metric tons,pounds per megawatthour,short tons,pounds per megawatthour,short tons
4997,2004,AZ,Arizona,PET,Petroleum,None,40,None,58,None,25,pounds per megawatthour,thousand metric tons,pounds per megawatthour,short tons,pounds per megawatthour,short tons
4998,2004,AZ,Arizona,OTH,Other,None,0,None,227,None,0,pounds per megawatthour,thousand metric tons,pounds per megawatthour,short tons,pounds per megawatthour,short tons


In [71]:
state_emissions[(state_emissions['period'] == '2023') & (state_emissions['stateid'] == 'CA')]

,period,stateid,stateDescription,fuelid,fuelDescription,co2-rate-lbs-mwh,co2-thousand-metric-tons,nox-rate-lbs-mwh,nox-short-tons,so2-rate-lbs-mwh,so2-short-tons,co2-rate-lbs-mwh-units,co2-thousand-metric-tons-units,nox-rate-lbs-mwh-units,nox-short-tons-units,so2-rate-lbs-mwh-units,so2-short-tons-units
240,2023,CA,California,ALL,Total,440,43360,.6,65559,0,1043,pounds per megawatthour,thousand metric tons,pounds per megawatthour,short tons,pounds per megawatthour,short tons
241,2023,CA,California,PET,Petroleum,None,68,None,588,None,85,pounds per megawatthour,thousand metric tons,pounds per megawatthour,short tons,pounds per megawatthour,short tons
242,2023,CA,California,OTH,Other,None,656,None,23494,None,558,pounds per megawatthour,thousand metric tons,pounds per megawatthour,short tons,pounds per megawatthour,short tons
243,2023,CA,California,NG,Natural Gas,None,41527,None,40742,None,229,pounds per megawatthour,thousand metric tons,pounds per megawatthour,short tons,pounds per megawatthour,short tons
244,2023,CA,California,COL,Coal,None,1109,None,735,None,171,pounds per megawatthour,thousand metric tons,pounds per megawatthour,short tons,pounds per megawatthour,short tons


In [59]:
# Crude oil import data
URL = 'https://api.eia.gov/v2/crude-oil-imports/data/?frequency=monthly&data[0]=quantity&start=2023-01&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000'
data = requests.get(URL, params={'api_key': key}).json()
crude_oil = pd.DataFrame.from_dict(data['response']['data'])

# Save to CSV
crude_oil.to_csv('crude_oil_imports.csv', index=False)


In [61]:
# Electric power operational data
URL = 'https://api.eia.gov/v2/electricity/electric-power-operational-data/data/?frequency=monthly&data[0]=ash-content&data[1]=consumption-for-eg&data[2]=consumption-for-eg-btu&data[3]=consumption-uto&data[4]=consumption-uto-btu&data[5]=cost&data[6]=cost-per-btu&data[7]=generation&data[8]=heat-content&data[9]=receipts&data[10]=receipts-btu&data[11]=stocks&data[12]=sulfur-content&data[13]=total-consumption&data[14]=total-consumption-btu&start=2001-01&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000'
data = requests.get(URL, params={'api_key': key}).json()
electricity_power_consumption = pd.DataFrame.from_dict(data['response']['data'])

# Save to CSV
electricity_power_consumption.to_csv('electricity_power_operational_data.csv', index=False)


,period,location,stateDescription,sectorid,sectorDescription,fueltypeid,fuelTypeDescription,ash-content,ash-content-units,consumption-for-eg,...,receipts-btu,receipts-btu-units,stocks,stocks-units,sulfur-content,sulfur-content-units,total-consumption,total-consumption-units,total-consumption-btu,total-consumption-btu-units
0,2024-09,NE,Nebraska,99,All Sectors,BIO,biomass,None,percent,171.037,...,None,billion Btu,None,thousand physical units,None,percent,173.97,thousand physical units,.08889,million MMBtu
1,2024-09,US,U.S. Total,1,Electric Utility,HPS,hydro-electric pumped storage,None,percent,0,...,None,billion Btu,0,thousand physical units,None,percent,0,thousand physical units,0,million MMBtu
2,2024-09,US,U.S. Total,1,Electric Utility,HYC,conventional hydroelectric,None,percent,0,...,None,billion Btu,0,thousand physical units,None,percent,0,thousand physical units,50.83491,million MMBtu
3,2024-09,WNC,West North Central,99,All Sectors,PET,petroleum,None,percent,None,...,251.53341,billion Btu,None,thousand barrels,None,percent,None,thousand barrels,None,million MMBtu
4,2024-09,WNC,West North Central,99,All Sectors,RC,refined coal,8.5,percent,1053.492,...,72.936,billion Btu,None,thousand short tons,.65,percent,1100.837,thousand short tons,16.55815,million MMBtu
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2024-09,NE,Nebraska,99,All Sectors,DFO,distillate fuel oil,None,percent,None,...,None,billion Btu,None,thousand short tons,None,percent,None,thousand short tons,None,million MMBtu
4996,2024-09,NE,Nebraska,99,All Sectors,COW,all coal products,5.33,percent,984.884,...,13288.08831,billion Btu,None,thousand short tons,.29,percent,1014.128,thousand short tons,17.56649,million MMBtu
4997,2024-09,NE,Nebraska,99,All Sectors,COL,"coal, excluding waste coal",5.33,percent,984.884,...,13288.08831,billion Btu,None,thousand short tons,.29,percent,1014.128,thousand short tons,17.56649,million MMBtu
4998,2024-09,NE,Nebraska,99,All Sectors,BIT,bituminous coal,None,percent,0,...,None,billion Btu,None,thousand short tons,None,percent,0,thousand short tons,0,million MMBtu
